Autor: Átila

In [3]:
import numpy as np # Álgebra linear
import pandas as pd # Processamento de dados
# import seaborn as sns # Visualização de dados
import matplotlib.pyplot as plt # Visualização de dados
from sklearn.preprocessing import StandardScaler # Normalização dos dados
from sklearn.preprocessing import OneHotEncoder # Codificação de atributos categóricos
from copy import deepcopy # Copiar objetos
import glob

In [6]:
nomes_pocos = ['merged_1-BRSA-551-SE', 'merged_1-BRSA-574-SE', 'merged_1-BRSA-595-SE', 'merged_1-BRSA-605-SE', 'merged_1-BRSA-659-SE',
              'merged_1-BRSA-689-SE', 'merged_1-BRSA-696-SE', 'merged_1-BRSA-698-SE']

# Criação de um dicionário para armazenar os DataFrames associados aos nomes dos poços
dfs_por_poco = {}

for poco in glob.glob(r'**/Dados-fusao' + "/*.csv", recursive=True):
    # Leitura do poço e armazenamento no dicionário
    df = pd.read_csv(f'{poco}')
    parts = poco.split("-")
    dfs_por_poco[f"P_{parts[len(parts)-2]}"] = df

# Simplifica os nomes dos poços
for idx, poco in enumerate(nomes_pocos):
  parts = poco.split("-")
  nomes_pocos[idx] = f"P_{parts[len(parts)-2]}"

In [8]:
for poco in nomes_pocos:
    # Seleciona apenas as colunas especificadas para o DataFrame
    dfs_por_poco[poco] = dfs_por_poco[poco][['Poço', 'Profundidade', 'UnidadeF', 'UnidadeM', 'Litologia', 'BS', 'CAL', 'DCAL', 'GR', 'RESD', 'DT', 'RHOB',
                                             'DRHO', 'NPHI', 'PE']].copy()

    # Calcular o log na base 10 dos valores da coluna 'RESD'
    dfs_por_poco[poco].loc[:, 'Log10_RESD'] = np.log10(dfs_por_poco[poco]['RESD'])

    # Reordenar as colunas para que 'Log10 RESD' esteja imediatamente após 'RESD'
    cols = dfs_por_poco[poco].columns.tolist()
    resd_index = cols.index('RESD')
    cols.insert(resd_index + 1, cols.pop(cols.index('Log10_RESD')))
    dfs_por_poco[poco] = dfs_por_poco[poco][cols]

In [9]:
# Aplicação dos filtros

def filtra_coluna(df, coluna, min=None, max=None):
    indices_para_remover = []
    for index, row in df.iterrows():
        if (min is not None and row[coluna] <= min) or (max is not None and row[coluna] >= max):
            indices_para_remover.append(index)

    df.drop(index=indices_para_remover, inplace=True)
    return df

colunas = ['GR', 'NPHI', 'RHOB', 'DRHO', 'RESD', 'DCAL', 'PE', 'Litologia']
for poco in nomes_pocos:
    #print(f"{poco}:")
    #print(f"Número de amostra antes do filtro de nulos: {len(dfs_por_poco[poco])}")

    # Excluir as linhas que não possuem alguma curva de perfil
    dfs_por_poco[poco].dropna(subset=colunas, inplace=True)
    #print(f"Número de amostras depois do filtro de nulos: {len(dfs_por_poco[poco])}")

    # Aplica os filtros de perfis definidos pelos geólogos
    dfs_por_poco[poco] = filtra_coluna(dfs_por_poco[poco], 'DCAL', max=1.5, min=-1)
    dfs_por_poco[poco] = filtra_coluna(dfs_por_poco[poco], 'DRHO', min=-0.15, max=0.15)
    #print(f"Número de amostras depois do filtro de valores máximos e mínimos: {len(dfs_por_poco[poco])}")


In [10]:
csv_directory = 'Dados-filtrados/'

for poco in nomes_pocos:
    # Salvar o DataFrame como um arquivo CSV
    dfs_por_poco[poco].to_csv(f"{csv_directory}/{poco}.csv", index=False)